In [1]:
!git clone https://github.com/thaid27/fairness_project.git
!mv fairness_project/* .
!mv fairness_project/.* . 2>/dev/null  # Pour les fichiers cachés comme .gitignore (ignore les erreurs)
!rm -rf fairness_project

Cloning into 'fairness_project'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 19 (delta 3), reused 16 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 25.79 MiB | 41.46 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [2]:
!pip install -q open_clip_torch
!pip install -q --upgrade optimum
!pip install -q bitsandbytes
!pip install -q transformers==4.46.3

!pip install -q segment_anything
!pip install -q git+https://github.com/facebookresearch/segment-anything.git
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 8.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━

In [3]:
from pathlib import Path
import os, shutil, random, json, math, cv2, torch, numpy as np, matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm


# CONFIG
DATASET_ROOT = Path("/kaggle/working")
OUTPUT_ROOT = Path("/kaggle/working/nyu_output")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

zip_path = "/kaggle/working/nyu.zip"

# Dézipper le contenu dans DATASET_ROOT
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(DATASET_ROOT)

INPAINT_PROMPT = "natural looking background consistent with the surrounding scene"
OBJECT_DETECTION_PROMPT = "Mark all the objects in this scene. Return only the coordinates"
NUM_INPAINTING_PER_OBJECT = 3

device0 = torch.device("cuda:0")
device1 = torch.device("cuda:1")

In [13]:
torch.cuda.empty_cache()

In [4]:
# Importation des modèles
from transformers import CLIPProcessor, CLIPModel, AutoModelForCausalLM, AutoProcessor
import timm
from segment_anything import SamPredictor, sam_model_registry
from diffusers import DiffusionPipeline

# Load Molmo
molmo_processor = AutoProcessor.from_pretrained(
    "cyan2k/molmo-7B-D-bnb-4bit",
    trust_remote_code=True,
    device_map="auto",
    torch_dtype="auto",
)

molmo_model = AutoModelForCausalLM.from_pretrained(
    "cyan2k/molmo-7B-D-bnb-4bit",
    trust_remote_code=True,
    offload_folder="offload",
    torch_dtype="auto",
).to(device0)



# Load SAM
sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h_4b8939.pth")
sam_predictor = SamPredictor(sam)

# Load StableDiffusion inpainting
pipe_stable_diffusion = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-inpainting").to(device1)

# Load CLIP
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device1)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load DINO
dino_model = timm.create_model('vit_small_patch8_224_dino', pretrained=True).to(device0)
dino_model.eval()

2025-06-14 21:46:36.483477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749937596.656018      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749937596.703705      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/95.9k [00:00<?, ?B/s]

Could not locate the preprocessing_molmo.py inside cyan2k/molmo-7B-D-bnb-4bit.


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like cyan2k/molmo-7B-D-bnb-4bit is not the path to a directory containing a file named preprocessing_molmo.py.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [6]:
# Dataloader function

def load_triplet(folder: Path):
    """Return PIL images: rgb, mask_A (1st), mask_B (2nd)."""
    rgb   = Image.open(folder + "/" + "img.jpg").convert("RGB")
    maskA = Image.open(folder + "/" + "A.png").convert("L")
    maskB = Image.open(folder + "/" + "B.png").convert("L")
    return rgb, maskA, maskB


In [10]:
import sys
sys.path.append('/kaggle/working')
from utils.molmo import points_object_detection_molmo, draw_points_on_image
from utils.sam import generate_masks_sam, display_mask
from utils.diversity_score import combined_score
from utils.inpainting import inpainting_for_scoring, inpainting_remove_object

def pipeline_global_1_image(image_path, display=False):
  # Load images
  image, maskA, maskB = load_triplet(image_path)
  output_path = "nyu_output" + "/" + "".join(image_path.split("/")[1:])

  # Get points for each object with molmo
  object_points = points_object_detection_molmo(molmo_processor, molmo_model, image, OBJECT_DETECTION_PROMPT)
  remaining_object_points = object_points

  if display:
    print(f"Output path: {output_path}")
    print(f"Numbers of objects: {len(object_points)}")
    print(f"Object coordinates: {object_points}")
    points_image = draw_points_on_image(image, object_points)
    display(points_image)


  # For each object in the picture
  for i in range(len(object_points)):

    # Get the masks of the remaining object
    masks, mask_points = generate_masks_sam(sam_predictor, image, remaining_object_points) # (mask, coordinates of the origin point from molmo)

    if display:
      print(f"\nIteration {i}:")
      print(f"Remaining object points: {remaining_object_points}:")
      display_mask(masks)

    # Calculate diversity score for each mask
    combined_score_list = []

    for mask in masks:
      # generate inpaiting for calculate the divesity score
      inpainted_images = []

      for _ in range(NUM_INPAINTING_PER_OBJECT):
        inpainted_images.append(inpainting_for_scoring(image, mask, pipe_stable_diffusion))

      combined_score_list.append(combined_score(inpainted_images, image))

    # Find the max diversity object
    max_diversity_mask = combined_score_list.max()
    max_diversity_idx = combined_score_list.argmax()

    if display:
      print(f"Diversity score list: {combined_score_list}")
      print(f"Max diversity object: {max_diversity_mask}")

    # Remove this object via inpainting
    inpainted_image = inpainting_remove_object(image, max_diversity_mask, pipe_stable_diffusion, INPAINT_PROMPT, output_path+ f"object_removal_{i}.png")
    if display:
      display(inpainted_image)

    # Remove coordinates from the removed object from remaining_object_points
    remaining_object_points.remove(remaining_object_points[max_diversity_idx])







In [15]:
pipeline_global_1_image("nyu/1001_0",display=True)

📝 Molmo output:
  <points x1="10.0" y1="60.4" x2="10.2" y2="65.3" x3="12.4" y3="58.7" x4="13.2" y4="56.2" x5="17.0" y5="56.2" x6="17.0" y6="58.7" x7="17.0" y7="60.4" x8="17.0" y8="62.3" x9="17.0" y9="64.0" x10="17.0" y10="65.3" x11="17.0" y11="67.0" x12="17.0" y12="68.7"
📍 Points détectés : []
Output path: nyu_output/1001_0
Numbers of objects: 0
Object coordinates: []


TypeError: 'bool' object is not callable